# Data Visualization Exam

In [ ]:
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px
import numpy as np

from zipfile import ZipFile
import shutil

## Data acquisition
CSV file creation from raw data and loading the CSV file into the program.

In [ ]:
dataFolderName = "data"
fileName = dataFolderName + '/DCSC_RACLI_01092021113430630.csv'
df = pd.read_csv(fileName)      # load data from CSV to program
df.head() # data loaded

## Data parsing

In [ ]:
# TODO: Check for errors in data
#       total is equal to the arithmetic mean of the parts? not seem
#       find missing data
#   	check that value apprendista < operaio < dirigente (for territory)

print('\nBefore remove duplicates: ' + str(len(df)) + ' rows')
df.drop_duplicates()
print('After remove duplicates:  ' + str(len(df)) + ' rows')

# are values reasonable?
print('\nMin value is ' + str(df['Value'].min()))
print('Max value is ' + str(df['Value'].max()))

# TODO: Change type

# TODO: Choose the level for hierachical data


# TODO: Transform the data
df['Territorio'] = df['Territorio'].str.replace(' / ','/')

## Data filtering

In [ ]:
df2 = df.copy()

# unique data
del df2['TIPO_DATO7'] # always the same (HOUWAG_ENTEMP_AV_MI)
del df2['Tipo dato']  # always the same (Retribuzione lorda oraria per ora retribuita delle posizioni lavorative dipendenti in euro (media).)

# ridondance of information
df2 = df2.drop(['SEXISTAT1', 'ETA1_A','PROFILO_PROF','CLLVT','Seleziona periodo'], axis=1)
# del df2['ATECO_2007']

df2 = df2[df2['Flag Codes'] != 'c'] # delete incomplete data

del df2['Flags']
del df2['Flag Codes']

df2.head()

## Data mining

In [ ]:
# granularity of sectors exists only for entire Italy (no territorial granularity)
df_sectors = df2.query('`Ateco 2007`!="TOTALE"')

# choose granularity of sectors
df_sectors = df_sectors.query('`ATECO_2007`>="A" & `ATECO_2007`<="Z"')

df_sectors = df_sectors.drop(['Territorio', 'ATECO_2007'], axis=1)

In [ ]:
df_territory = df2.query('`Ateco 2007`=="TOTALE"')
df_territory = df_territory.drop(['Ateco 2007', 'ATECO_2007'], axis=1)

In [ ]:
# TODO: start with univariate analysis (one variable at a time), continue with multivariate analysis

## Question 1
Do women earn less than men in Italy in private companies? Where is the most difference?

In [ ]:
df_sex = df_territory.query('Sesso!="totale"')
df_sex = df_sex.drop(['Classe di età','Qualifica contrattuale','Classe di dipendenti'],axis=1)
len(df_sex)

### Plot line chart

In [ ]:
""" df_sex_tot = df_sex.query('Territorio=="Italia"')

title = 'Gender difference salary in Italy'
labels = ['Men','Women']
colors = ['rgb(67,67,67)', 'rgb(115,115,115)']
mode_size = [8, 8]
line_size = [2, 2]

x_data = np.vstack((np.arange(df_sex_tot.TIME.min(), df_sex_tot.TIME.max()+1),)*2)

df_sex_tot.sort_values(by='TIME')
y_tot_m = df_sex_tot.query('Sesso=="maschi"')
y_tot_w = df_sex_tot.query('Sesso=="femmine"')
y_data = np.array([y_tot_m['Value'],y_tot_w['Value']])

fig = go.Figure()

for i in range(0, 2):
    fig.add_trace(go.Scatter(x=x_data[i], y=y_data[i], mode='lines',
        name=labels[i],
        line=dict(color=colors[i], width=line_size[i]),
        connectgaps=True,
    ))

    # endpoints
    fig.add_trace(go.Scatter(
        x=[x_data[i][0], x_data[i][-1]],
        y=[y_data[i][0], y_data[i][-1]],
        mode='markers',
        marker=dict(color=colors[i], size=mode_size[i])
    ))

fig.update_layout(
    xaxis=dict(
        showline=True,
        showgrid=False,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)'
        )
    ),
    yaxis=dict(
        showgrid=True,
        zeroline=True,
        showline=True,
        showticklabels=True,
        range = [0, max(df_sex_tot['Value'])]
    ),
    autosize=False,
    margin=dict(
        autoexpand=False,
        l=100,
        r=20,
        t=110,
    ),
    showlegend=False,
    plot_bgcolor='white'
)

annotations = []

# Adding labels
for y_trace, label, color in zip(y_data, labels, colors):
    # labeling the left_side of the plot
    annotations.append(dict(xref='paper', x=0.05, y=y_trace[0],
                                  xanchor='right', yanchor='middle',
                                  text=label + ' {}'.format(y_trace[0]),
                                  font=dict(family='Arial',
                                            size=16),
                                  showarrow=False))
    # labeling the right_side of the plot
    annotations.append(dict(xref='paper', x=0.95, y=y_trace[2],
                                  xanchor='left', yanchor='middle',
                                  text='{}'.format(y_trace[2]),
                                  font=dict(family='Arial',
                                            size=16),
                                  showarrow=False))
# Title
annotations.append(dict(xref='paper', yref='paper', x=0.0, y=1.05,
                              xanchor='left', yanchor='bottom',
                              text=title,
                              font=dict(family='Arial',
                                        size=30,
                                        color='rgb(37,37,37)'),
                              showarrow=False))
# Source
annotations.append(dict(xref='paper', yref='paper', x=0.5, y=-0.1,
                              xanchor='center', yanchor='top',
                              text='Source: PewResearch Center & ' +
                                   'Storytelling with data',
                              font=dict(family='Arial',
                                        size=12,
                                        color='rgb(150,150,150)'),
                              showarrow=False))

fig.update_layout(annotations=annotations)

fig.show() """

In [ ]:
""" fig = go.Figure()

for i in range(0, 2):
    fig.add_trace(go.Scatter(x=x_data[i], y=y_data[i], mode='lines',
        name=labels[i],
        line=dict(color=colors[i], width=line_size[i]),
        connectgaps=True,
    ))

    # endpoints
    fig.add_trace(go.Scatter(
        x=[x_data[i][0], x_data[i][-1]],
        y=[y_data[i][0], y_data[i][-1]],
        mode='markers',
        marker=dict(color=colors[i], size=mode_size[i])
    ))

fig.update_layout(
    xaxis=dict(
        showline=True,
        showgrid=False,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        ),
    ),
    yaxis=dict(
        showgrid=False,
        zeroline=False,
        showline=False,
        showticklabels=False,
    ),
    autosize=False,
    margin=dict(
        autoexpand=False,
        l=100,
        r=20,
        t=110,
    ),
    showlegend=False,
    plot_bgcolor='white'
)

annotations = []

# Adding labels
for y_trace, label, color in zip(y_data, labels, colors):
    # labeling the left_side of the plot
    annotations.append(dict(xref='paper', x=0.05, y=y_trace[0],
                                  xanchor='right', yanchor='middle',
                                  text=label + ' {}'.format(y_trace[0]),
                                  font=dict(family='Arial',
                                            size=16),
                                  showarrow=False))
    # labeling the right_side of the plot
    annotations.append(dict(xref='paper', x=0.95, y=y_trace[2],
                                  xanchor='left', yanchor='middle',
                                  text='{}'.format(y_trace[2]),
                                  font=dict(family='Arial',
                                            size=16),
                                  showarrow=False))
# Title
annotations.append(dict(xref='paper', yref='paper', x=0.0, y=1.05,
                              xanchor='left', yanchor='bottom',
                              text=title,
                              font=dict(family='Arial',
                                        size=30,
                                        color='rgb(37,37,37)'),
                              showarrow=False))
# Source
annotations.append(dict(xref='paper', yref='paper', x=0.5, y=-0.1,
                              xanchor='center', yanchor='top',
                              text='Source: PewResearch Center & ' +
                                   'Storytelling with data',
                              font=dict(family='Arial',
                                        size=12,
                                        color='rgb(150,150,150)'),
                              showarrow=False))

fig.update_layout(annotations=annotations)

fig.show() """

### Plot gender map

In [ ]:
import matplotlib.pyplot as plt
import geopandas as gpd
from geopandas import GeoDataFrame

In [ ]:
""" map_df = {}
for year in range(2014,2018):
    year1=2017 
    if year==2014: year1=2014
    fp = f'{dataFolderName}/province_shapes/ProvCM0101{year1}_g/ProvCM0101{year1}_g_WGS84.shp'
    
    map_df[f"map_{year}"] = gpd.read_file(fp) #reading the file stored in variable fp
    # print(map_df[f"map_{year}"].plot())

# map_df['map_2015']
 """

In [ ]:
# consider only province data
df_sex_province = df_sex[df_sex['ITTER107'].str.contains('.{5}')]

df_sex_province_year = {}
for year in range(2014,2018):
    temp = df_sex_province.query(f'TIME=={year}')
    temp_f = temp.query('Sesso=="maschi"').set_index('ITTER107').drop('Sesso',axis=1)
    temp_d = temp.query('Sesso=="femmine"').set_index('ITTER107')
    temp_f['Value'] = [temp_f['Value'][prov]-temp_d['Value'][prov] for prov in temp['ITTER107'].drop_duplicates()]
    df_sex_province_year[f'group_{year}'] = temp_f

del temp
del temp_d
del temp_f
# df_sex_province_year["group2014"]

In [ ]:
""" # join for connect ITTER107 to SIGLA
csv_prov = pd.DataFrame(columns=['sigla','codice'])

fileName = "province_statistical_codes/Codici-statistici-e-denominazioni-al-01_01_201"
for anno in (4,5,6,7):
    temp = pd.read_csv("data/"+fileName + str(anno)+'.csv',sep=';',encoding='ANSI', na_filter=False)
    temp = temp[['Sigla automobilistica',temp.columns[-1]]].drop_duplicates()
    temp = temp.rename(columns={temp.columns[0]:csv_prov.columns[0],temp.columns[1]:csv_prov.columns[1]})
    csv_prov = csv_prov.append(temp).drop_duplicates()
    csv_prov.append({'sigla':'MB','codice':'IT108'},ignore_index=True)
    csv_prov.append({'sigla':'FM','codice':'IT109'},ignore_index=True)
    csv_prov.append({'sigla':'BT','codice':'IT110'},ignore_index=True) """

In [ ]:
""" merged1 = df_sex_province_year["group_2017"].join(csv_prov.set_index(csv_prov.columns[-1]))
merged1.sort_values(by='sigla')#query('Territorio=="Fermo"')
merged1[merged1.isnull().any(axis=1)].drop_duplicates() """

In [ ]:
""" merged = {}
for year in range(2014,2018):
    merged[f'merged_{year}'] = df_sex_province_year[f'group_{year}'].set_index('Territorio').join(
        map_df[f'map_{year}'].set_index('DEN_PROV' if year==2014 else 'DEN_PCM'))
    if year==2015: print(merged[f'merged_{year}'][merged[f'merged_{year}'].isnull().any(axis=1)].drop_duplicates()) """

In [ ]:
""" # set a variable that will call whatever column we want to visualise on the map
variable = 'TIME'#'Value'
# set the range for the choropleth
vmin, vmax = 120, 220
# create figure and axes for Matplotlib
fig, ax = plt.subplots(1, figsize=(10, 6))

# for year in range(2014,2018):
#     merged[f'merged_{year}'] = GeoDataFrame(merged[f'merged_{year}'])
#     merged[f'merged_{year}'].plot(column=variable, cmap='BuGn', linewidth=0.01, ax=ax, edgecolor='0.8')

merged['merged_2015'] = GeoDataFrame(merged['merged_2015'])
merged['merged_2015'].plot(column=variable, cmap='hsv', linewidth=1, ax=ax, edgecolor='0.1') """